<a href="https://colab.research.google.com/github/Viktoriia-kama/ML_hw4-hw16/blob/main/HW10_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from surprise import Dataset
from surprise import SVD, SVDpp, NMF
from surprise.model_selection import cross_validate, GridSearchCV

In [20]:
!pip install surprise

In [21]:
data = Dataset.load_builtin('ml-100k')
data

In [22]:
models = {
    "SVD": SVD(),
    "SVD++": SVDpp(),
    "NMF": NMF()
}

results = {}

for name, model in models.items():
  print(f"Evaluating {name}...")
  results[name] = cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating SVD...
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9385  0.9389  0.9343  0.9370  0.9379  0.9373  0.0016  
MAE (testset)     0.7371  0.7412  0.7352  0.7374  0.7424  0.7387  0.0027  
Fit time          1.98    1.51    1.52    1.46    4.58    2.21    1.20    
Test time         0.31    0.12    0.20    0.24    0.25    0.22    0.06    
Evaluating SVD++...
Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9239  0.9124  0.9189  0.9270  0.9143  0.9193  0.0055  
MAE (testset)     0.7236  0.7166  0.7210  0.7265  0.7173  0.7210  0.0038  
Fit time          29.09   28.74   27.28   27.22   27.22   27.91   0.83    
Test time         4.81    4.96    5.57    4.81    5.76    5.18    0.40    
Evaluating NMF...
Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold

In [23]:
summary = {}

for name, res in results.items():
  summary[name] = {
       'RMSE Mean': res['test_rmse'].mean(),
        'RMSE Std': res['test_rmse'].std(),
        'MAE Mean': res['test_mae'].mean(),
        'MAE Std': res['test_mae'].std()
  }

results_df = pd.DataFrame(summary).T
results_df

,RMSE Mean,RMSE Std,MAE Mean,MAE Std
SVD,0.937305,0.001642,0.738664,0.002695
SVD++,0.919272,0.005532,0.721017,0.003761
NMF,0.963935,0.003059,0.757717,0.002715


В цьому прикладі, модель SVD++ має найменше середнє значення RMSE та MAE, що означає, що вона найточніша серед трьох. Вона також має найменші значення стандартного відхилення, що означає, що вона найбільш стабільна. Тому, ймовірно, SVD++ є найкращим вибором серед цих трьох моделей.

In [26]:
# Налаштування параметрів для SVD
param_grid = {
    'n_factors': [50, 100, 150],
    'n_epochs': [20, 30, 40],
    'lr_all': [0.002, 0.005, 0.01],
    'reg_all': [0.02, 0.05, 0.1]
}

gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=5)
gs.fit(data)

# Виведення найкращих параметрів та результатів
print("Найкращі параметри:", gs.best_params['rmse'])
print("Найкращий RMSE:", gs.best_score['rmse'])

# Найкраща модель
best_model = gs.best_estimator['rmse']


KeyboardInterrupt: 

Не певна з якої причини цей код виконувався більше 15хв, але так і не показав результат. Уявімо, що ми отримали такі параметри:
    'n_factors': 100,
    'n_epochs': 30,
    'lr_all': 0.005,
    'reg_all': 0.02
Далі я тестую мою модель SVD на цих параметрах

In [33]:
optimal_params = {
    'n_factors': 100,
    'n_epochs': 30,
    'lr_all': 0.005,
    'reg_all': 0.02
}
model = SVD(**optimal_params)
trainset = data.build_full_trainset()
model.fit(trainset)

models = {
    "SVD": model,
    "SVD++": SVDpp(),
    "NMF": NMF()
}

results1 = {}

for name, model in models.items():
  print(f"Evaluating {name}...")
  results1[name] = cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)



Evaluating SVD...
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9417  0.9451  0.9428  0.9548  0.9369  0.9443  0.0059  
MAE (testset)     0.7402  0.7421  0.7403  0.7463  0.7384  0.7415  0.0027  
Fit time          2.39    2.67    2.86    2.18    2.18    2.46    0.27    
Test time         0.17    0.44    0.12    0.28    0.12    0.22    0.12    
Evaluating SVD++...
Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9162  0.9176  0.9235  0.9204  0.9230  0.9201  0.0029  
MAE (testset)     0.7200  0.7215  0.7231  0.7209  0.7237  0.7219  0.0014  
Fit time          29.08   28.89   30.04   28.99   28.39   29.08   0.54    
Test time         5.97    5.57    5.27    5.94    5.75    5.70    0.26    
Evaluating NMF...
Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold

In [34]:
summary1 = {}
for name, res in results1.items():
  summary1[name] = {
        'RMSE Mean': np.mean(res['test_rmse']),
        'RMSE Std': np.std(res['test_rmse']),
        'MAE Mean': np.mean(res['test_mae']),
        'MAE Std': np.std(res['test_mae']),
    }

results_df1 = pd.DataFrame(summary1).T
results_df1

,RMSE Mean,RMSE Std,MAE Mean,MAE Std
SVD,0.944275,0.005890,0.741475,0.002671
SVD++,0.920137,0.002863,0.721861,0.001369
NMF,0.965910,0.002291,0.759407,0.002214


Порівнявши результати метрик, роблю висновок, що мої optimal_params не є найкращими. І не дивно, бо довелось взяти їх на око

У результаті, найбільш оптимальним алгоритмом є SVD++, оскільки його метрики є найнижчими